In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

In [2]:
with sqlite3.connect('dataset/Cleaned.db') as conn:
    train = pd.read_sql_query('SELECT * FROM train', conn)

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
labels = train.is_duplicate
y_true = list(map(int, labels))

In [5]:
Xtrain, Xtest, ytrain, ytest = train_test_split(train, y_true, stratify=y_true, test_size=0.3)

In [6]:
Xtrain.shape

(283002, 46)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
tfidf = TfidfVectorizer(stop_words='english')

In [9]:
from pyemd import emd
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/home/paperspace/w2v/GoogleNews-vectors-negative300.bin',
                                         binary=True)

In [10]:
def sent2vec(sentence, model, method='tfidf', **kwargs):
    """
    Generic function to convert a sentence to a vector using
    avg or TFIDF vecorization.
    
    :param sentence: Sentence to be converted.
    :param model: The word2vec model
    """
    
    ##### It is recommended to pass seperate stopwords #####
    stopwords = kwargs.get('stopwords')
    if stopwords is None:
        from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
        stopwords = ENGLISH_STOP_WORDS
    
    ##### It is recommended to pass seperate tokenizers #####
    tokenizer = kwargs.get('tokenizer')
    if tokenizer is None:
        from nltk.tokenize import RegexpTokenizer
        tokenizer = RegexpTokenizer(r'\w+')

    words = tokenizer.tokenize(sentence) # Tokenize the words
    words = {each for each in words if each not in stopwords} # Remove all the stopwords
    
    V = []
    
    for word in words: # Process over all the words in the sentence
        if model.__contains__(word):
            V.append(model[word])
    V = np.array(V)
    
    # If no words were present in the model
    # or blank sentence was passed, return a
    # word vector with all 0's
    if V.shape[0] == 0:
        # If model returns word2vec of different size
        # Default value is taken 300
        custom_shape = kwargs.get('shape', 300)
        return np.zeros(custom_shape)
    
    # If there is atleast one word in the sentence that
    # was vectoried properly
    
    if method.lower() == 'avg':
        V = V.sum(axis=0)
        return V / np.sqrt((V ** 2).sum())
    
    elif method.lower() == 'tfidf':
        tfidf_model = kwargs.get('tfidf_model') # Load the tfidf model
        if tfidf_model: # If model loaded sucessfully
            tfidf_vec = tfidf_model.transform([sentence]) # get TFIDF for the sentence
            indx = tfidf_model.vocabulary_.get(word, -1)
            tfidfs = []
            for word in words:
                if model.__contains__(word):
                    if indx != -1:
                        tfidfs.append(tfidf_vec[0, indx])
                    else:
                        tfidfs.append(0.0)
            tfidfs = np.array(tfidfs)
            denominator = tfidfs.sum()
            if denominator == 0.0: # No word is representred in tfidf and w2v both
                # Better than skipping that sentence
                denominator = tfidf_model.idf_.min() * 0.01
            numerator = V * tfidfs.reshape(V.shape[0], 1)
            numerator = numerator.sum(axis=0)
            return numerator / denominator
        else:
            raise ValueError('No tfidf model is present')

In [11]:
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
tokenizer = RegexpTokenizer(r'\w+')

In [12]:
tfidf.fit(Xtrain.question1)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
question1_vectors_train = np.zeros((Xtrain.shape[0], 300))
for i, q in tqdm(enumerate(Xtrain.question1.values), total=283002):
    question1_vectors_train[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

100%|██████████| 283002/283002 [03:30<00:00, 1345.50it/s]


In [14]:
question1_vectors_test = np.zeros((Xtest.shape[0], 300))
for i, q in tqdm(enumerate(Xtest.question1.values), total=121287):
    question1_vectors_test[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

100%|██████████| 121287/121287 [01:30<00:00, 1337.40it/s]


In [15]:
tfidf.fit(Xtrain.question2)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
question2_vectors_train = np.zeros((Xtrain.shape[0], 300))
for i, q in tqdm(enumerate(Xtrain.question2.values), total=283002):
    question2_vectors_train[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

100%|██████████| 283002/283002 [03:29<00:00, 1348.75it/s]


In [17]:
question2_vectors_test = np.zeros((Xtest.shape[0], 300))
for i, q in tqdm(enumerate(Xtest.question2.values), total=121287):
    question2_vectors_test[i, :] = sent2vec(q, model=model, tfidf_model=tfidf,
                                       tokenizer=tokenizer, stopwords=ENGLISH_STOP_WORDS)

100%|██████████| 121287/121287 [01:30<00:00, 1342.30it/s]


In [18]:
question1_vectors_train.shape

(283002, 300)

In [19]:
question2_vectors_train.shape

(283002, 300)

In [20]:
Xtrain.drop([
    'id',
    'qid1',
    'qid2',
    'question1',
    'question2',
    'is_duplicate'
], axis=1, inplace=True)

/home/paperspace/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [ ]:
Xtest.drop([
    'id',
    'qid1',
    'qid2',
    'question1',
    'question2',
    'is_duplicate'
], axis=1, inplace=True)

In [22]:
X_train = np.hstack((question1_vectors_train, question2_vectors_train, np.array(Xtrain)))
X_test = np.hstack((question1_vectors_test, question2_vectors_test, np.array(Xtest)))

In [23]:
X_train.shape

(283002, 640)

In [24]:
X_test.shape

(121287, 640)

In [4]:
import xgboost as xgb
from sklearn.metrics import log_loss

In [7]:
params = {
    'max_depth': [2, 4, 6],  # How deep the base learners need to go -- typically small value
    'subsample': [0.5, 0.75, 1.0], # Row sampling like Random Forest 
    'colsample_bytree': [0.3, 0.5, 0.7, 1.0]  # Column sampling like Random Forest
}


In [49]:
estimator = xgb.XGBClassifier(random_state=42, objective='binary:logistic',  n_jobs=-1, n_estimators=400)

In [10]:
!ls dataset

Cleaned.db	       test.csv    w2v_test.npz.npy   w2v_xtrain.npy
question1_vectors.npy  train.csv   w2v_train.npz.npy  w2v_ytest.npy
question2_vectors.npy  W2VData.db  w2v_xtest.npy      w2v_.ytrain.npy


In [36]:
np.save('dataset/w2v_train.npz', X_train)
np.save('dataset/w2v_test.npz', X_test)

In [2]:
X_train = np.load('dataset/w2v_xtrain.npy')
X_test = np.load('dataset/w2v_xtest.npy')
ytrain = np.load('dataset/w2v_.ytrain.npy')
ytest = np.load('dataset/w2v_ytest.npy')

In [3]:
X_train.shape

(283002, 640)

In [29]:
from sklearn.model_selection import StratifiedKFold

### Learning Rate & max_depth

In [5]:
learning_rates = [10, 1, 0.1, 0.01, 0.001]
depths = [2, 4, 6]

In [6]:
log_losses = []
for lr in learning_rates:
    for dep in depths:
        params = {}
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'logloss'
        params['eta'] = lr
        params['max_depth'] = dep

        d_train = xgb.DMatrix(X_train, label=ytrain)
        d_test = xgb.DMatrix(X_test, label=ytest)

        watchlist = [(d_train, 'train'), (d_test, 'valid')]

        clf = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=2)
        predict_y = clf.predict(d_test)
        log_losses.append(log_loss(ytest, predict_y, eps=1e-15))
        print(f"Done for lr = {lr} and depth = {dep}")

[06:15:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[0]	train-logloss:1.31199	valid-logloss:1.33122
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
[06:15:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:15:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[2]	train-logloss:9.71217	valid-logloss:9.80676
[06:15:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 4 extra nodes, 0 pruned nodes, max_depth=2
[06:15:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[4]	train-logloss:13.6019	valid-logloss:13.6021
[06:15:33] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0

/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1817: RuntimeWarning: divide by zero encountered in log
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)
/home/paperspace/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1817: RuntimeWarning: invalid value encountered in multiply
  loss = -(transformed_labels * np.log(y_pred)).sum(axis=1)


[06:15:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 30 extra nodes, 0 pruned nodes, max_depth=4
[0]	train-logloss:1.54953	valid-logloss:1.58352
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
[06:15:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 28 extra nodes, 0 pruned nodes, max_depth=4
[06:15:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 16 extra nodes, 0 pruned nodes, max_depth=4
[2]	train-logloss:10.8177	valid-logloss:10.9035
[06:15:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[06:15:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes, 0 pruned nodes, max_depth=0
[4]	train-logloss:23.2394	valid-logloss:23.2393
[06:15:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 0 extra nodes

[06:16:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:16:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[10]	train-logloss:0.394502	valid-logloss:0.398761
[06:16:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:16:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[12]	train-logloss:0.391462	valid-logloss:0.396043
[06:16:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:16:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[14]	train-logloss:0.388465	valid-logloss:0.393202
[06:16:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_d

[06:17:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:17:06] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[66]	train-logloss:0.354752	valid-logloss:0.363637
[06:17:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:17:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[68]	train-logloss:0.354041	valid-logloss:0.362987
[06:17:08] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[06:17:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[70]	train-logloss:0.353584	valid-logloss:0.362815
[06:17:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_d

...

[07:46:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:46:09] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[178]	train-logloss:0.621197	valid-logloss:0.621899
[07:46:12] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:46:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[180]	train-logloss:0.620515	valid-logloss:0.621224
[07:46:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:46:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[182]	train-logloss:0.619836	valid-logloss:0.620552
[07:46:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pr

[07:48:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[232]	train-logloss:0.603501	valid-logloss:0.604383
[07:48:18] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:48:21] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[234]	train-logloss:0.602876	valid-logloss:0.603763
[07:48:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:48:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[236]	train-logloss:0.602256	valid-logloss:0.60315
[07:48:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:48:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pru

[07:50:23] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:50:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[288]	train-logloss:0.58674	valid-logloss:0.587796
[07:50:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:50:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[290]	train-logloss:0.58617	valid-logloss:0.587231
[07:50:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:50:35] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[292]	train-logloss:0.585603	valid-logloss:0.58667
[07:50:37] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 prune

[07:52:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[342]	train-logloss:0.571966	valid-logloss:0.5732
[07:52:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:52:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[344]	train-logloss:0.571442	valid-logloss:0.572683
[07:52:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:52:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[346]	train-logloss:0.570915	valid-logloss:0.572161
[07:52:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:52:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 prun

[07:54:40] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[07:54:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[398]	train-logloss:0.557827	valid-logloss:0.559231
[07:54:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[399]	train-logloss:0.557584	valid-logloss:0.558991
Done for lr = 0.001 and depth = 6


In [13]:
np.argsort(log_losses)[0:4]

array([8, 7, 3, 6])

In [11]:
log_losses[8]

0.3193787284525697

In [14]:
comb = []
for lr in learning_rates:
    for dep in depths:
        comb.append((lr, dep))

In [16]:
comb[8]

(0.1, 6)

In [17]:
comb[7]

(0.1, 4)

In [18]:
comb[3]

(1, 2)

In [19]:
comb[6]

(0.1, 2)

It seems that 0.1 is the best learning rate. Let's fine tune the max depth which decreses the validation loss but still seems to overfit least

In [4]:
depths = [1, 2, 3, 4, 5, 6]

In [5]:
result = []
for dep in depths:
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    params['eta'] = 0.1
    params['max_depth'] = dep
    params['silent'] = 1
#     silent=1

    d_train = xgb.DMatrix(X_train, label=ytrain)
    d_test = xgb.DMatrix(X_test, label=ytest)

    watchlist = [(d_train, 'train'), (d_test, 'valid')]

    clf = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=1)
    predict_y = clf.predict(d_test)
    predict_y_train = clf.predict(d_train)
    res = (dep, log_loss(ytrain, predict_y_train, eps=1e-15), log_loss(ytest, predict_y, eps=1e-15))
    result.append(res)
    print(f"Done for depth = {dep}")

[0]	train-logloss:0.673618	valid-logloss:0.673765
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.656039	valid-logloss:0.656307
[2]	train-logloss:0.640972	valid-logloss:0.641336
[3]	train-logloss:0.626936	valid-logloss:0.627366
[4]	train-logloss:0.614588	valid-logloss:0.615113
[5]	train-logloss:0.603597	valid-logloss:0.604164
[6]	train-logloss:0.593643	valid-logloss:0.59433
[7]	train-logloss:0.584796	valid-logloss:0.585546
[8]	train-logloss:0.576358	valid-logloss:0.577205
[9]	train-logloss:0.568857	valid-logloss:0.569758
[10]	train-logloss:0.561705	valid-logloss:0.562708
[11]	train-logloss:0.555268	valid-logloss:0.556344
[12]	train-logloss:0.549315	valid-logloss:0.550467
[13]	train-logloss:0.543747	valid-logloss:0.544977
[14]	train-logloss:0.538629	valid-logloss:0.539952
[15]	train-logloss:0.533709	valid-logloss:0.535062
[16]	train-logloss:0.529051	valid-logloss:0.

[158]	train-logloss:0.412938	valid-logloss:0.416086
[159]	train-logloss:0.412773	valid-logloss:0.415928
[160]	train-logloss:0.412607	valid-logloss:0.41576
[161]	train-logloss:0.412445	valid-logloss:0.415589
[162]	train-logloss:0.412281	valid-logloss:0.415457
[163]	train-logloss:0.412118	valid-logloss:0.415283
[164]	train-logloss:0.411955	valid-logloss:0.415141
[165]	train-logloss:0.411797	valid-logloss:0.414987
[166]	train-logloss:0.411639	valid-logloss:0.414837
[167]	train-logloss:0.41148	valid-logloss:0.41469
[168]	train-logloss:0.411325	valid-logloss:0.414535
[169]	train-logloss:0.411163	valid-logloss:0.414375
[170]	train-logloss:0.411006	valid-logloss:0.414216
[171]	train-logloss:0.410852	valid-logloss:0.414063
[172]	train-logloss:0.4107	valid-logloss:0.413906
[173]	train-logloss:0.410548	valid-logloss:0.41375
[174]	train-logloss:0.410396	valid-logloss:0.413596
[175]	train-logloss:0.410243	valid-logloss:0.413444
[176]	train-logloss:0.410096	valid-logloss:0.413296
[177]	train-loglos

[317]	train-logloss:0.395909	valid-logloss:0.399514
[318]	train-logloss:0.395839	valid-logloss:0.399455
[319]	train-logloss:0.395768	valid-logloss:0.399381
[320]	train-logloss:0.395699	valid-logloss:0.399322
[321]	train-logloss:0.39563	valid-logloss:0.399263
[322]	train-logloss:0.395563	valid-logloss:0.399192
[323]	train-logloss:0.395492	valid-logloss:0.399126
[324]	train-logloss:0.395424	valid-logloss:0.399053
[325]	train-logloss:0.395358	valid-logloss:0.39899
[326]	train-logloss:0.39529	valid-logloss:0.398912
[327]	train-logloss:0.395223	valid-logloss:0.398842
[328]	train-logloss:0.395157	valid-logloss:0.398789
[329]	train-logloss:0.395091	valid-logloss:0.398731
[330]	train-logloss:0.395025	valid-logloss:0.398664
[331]	train-logloss:0.394953	valid-logloss:0.398596
[332]	train-logloss:0.394889	valid-logloss:0.398533
[333]	train-logloss:0.394824	valid-logloss:0.398459
[334]	train-logloss:0.394759	valid-logloss:0.398414
[335]	train-logloss:0.394694	valid-logloss:0.398338
[336]	train-log

[74]	train-logloss:0.397434	valid-logloss:0.400722
[75]	train-logloss:0.397151	valid-logloss:0.400442
[76]	train-logloss:0.396718	valid-logloss:0.400022
[77]	train-logloss:0.395703	valid-logloss:0.399005
[78]	train-logloss:0.395436	valid-logloss:0.398744
[79]	train-logloss:0.394811	valid-logloss:0.398113
[80]	train-logloss:0.394462	valid-logloss:0.397764
[81]	train-logloss:0.393814	valid-logloss:0.397124
[82]	train-logloss:0.393404	valid-logloss:0.396692
[83]	train-logloss:0.393052	valid-logloss:0.396351
[84]	train-logloss:0.392109	valid-logloss:0.395413
[85]	train-logloss:0.3918	valid-logloss:0.395099
[86]	train-logloss:0.391508	valid-logloss:0.394835
[87]	train-logloss:0.391256	valid-logloss:0.394597
[88]	train-logloss:0.390976	valid-logloss:0.394338
[89]	train-logloss:0.390714	valid-logloss:0.394093
[90]	train-logloss:0.390509	valid-logloss:0.393878
[91]	train-logloss:0.390252	valid-logloss:0.39364
[92]	train-logloss:0.390028	valid-logloss:0.39344
[93]	train-logloss:0.389511	valid-l

[233]	train-logloss:0.364434	valid-logloss:0.369276
[234]	train-logloss:0.364332	valid-logloss:0.369186
[235]	train-logloss:0.364274	valid-logloss:0.369124
[236]	train-logloss:0.364199	valid-logloss:0.369059
[237]	train-logloss:0.364129	valid-logloss:0.369007
[238]	train-logloss:0.364061	valid-logloss:0.368944
[239]	train-logloss:0.36397	valid-logloss:0.368873
[240]	train-logloss:0.363906	valid-logloss:0.368824
[241]	train-logloss:0.363776	valid-logloss:0.368703
[242]	train-logloss:0.363579	valid-logloss:0.368532
[243]	train-logloss:0.363529	valid-logloss:0.368497
[244]	train-logloss:0.363459	valid-logloss:0.368419
[245]	train-logloss:0.363339	valid-logloss:0.368316
[246]	train-logloss:0.363176	valid-logloss:0.368169
[247]	train-logloss:0.363096	valid-logloss:0.368099
[248]	train-logloss:0.362949	valid-logloss:0.367956
[249]	train-logloss:0.362873	valid-logloss:0.367882
[250]	train-logloss:0.362815	valid-logloss:0.36783
[251]	train-logloss:0.362772	valid-logloss:0.367794
[252]	train-lo

...

[142]	train-logloss:0.326069	valid-logloss:0.342789
[143]	train-logloss:0.325747	valid-logloss:0.342594
[144]	train-logloss:0.325583	valid-logloss:0.342488
[145]	train-logloss:0.325338	valid-logloss:0.342324
[146]	train-logloss:0.325066	valid-logloss:0.342116
[147]	train-logloss:0.324782	valid-logloss:0.341929
[148]	train-logloss:0.324553	valid-logloss:0.341793
[149]	train-logloss:0.324276	valid-logloss:0.34164
[150]	train-logloss:0.324012	valid-logloss:0.341498
[151]	train-logloss:0.323763	valid-logloss:0.341407
[152]	train-logloss:0.323471	valid-logloss:0.34118
[153]	train-logloss:0.323228	valid-logloss:0.341069
[154]	train-logloss:0.323105	valid-logloss:0.341011
[155]	train-logloss:0.32289	valid-logloss:0.340909
[156]	train-logloss:0.322645	valid-logloss:0.340788
[157]	train-logloss:0.322401	valid-logloss:0.34068
[158]	train-logloss:0.322135	valid-logloss:0.340543
[159]	train-logloss:0.321854	valid-logloss:0.34038
[160]	train-logloss:0.32152	valid-logloss:0.340151
[161]	train-loglos

[301]	train-logloss:0.296701	valid-logloss:0.330125
[302]	train-logloss:0.296546	valid-logloss:0.330085
[303]	train-logloss:0.296453	valid-logloss:0.330069
[304]	train-logloss:0.29639	valid-logloss:0.33006
[305]	train-logloss:0.296245	valid-logloss:0.33
[306]	train-logloss:0.29605	valid-logloss:0.329956
[307]	train-logloss:0.295871	valid-logloss:0.329874
[308]	train-logloss:0.295786	valid-logloss:0.329861
[309]	train-logloss:0.29563	valid-logloss:0.32984
[310]	train-logloss:0.295426	valid-logloss:0.329761
[311]	train-logloss:0.295284	valid-logloss:0.329713
[312]	train-logloss:0.295112	valid-logloss:0.329664
[313]	train-logloss:0.294934	valid-logloss:0.32957
[314]	train-logloss:0.294773	valid-logloss:0.329511
[315]	train-logloss:0.294608	valid-logloss:0.329468
[316]	train-logloss:0.2945	valid-logloss:0.32944
[317]	train-logloss:0.294387	valid-logloss:0.329412
[318]	train-logloss:0.294304	valid-logloss:0.329398
[319]	train-logloss:0.29414	valid-logloss:0.329336
[320]	train-logloss:0.2940

[58]	train-logloss:0.343812	valid-logloss:0.355998
[59]	train-logloss:0.343237	valid-logloss:0.355631
[60]	train-logloss:0.342646	valid-logloss:0.3552
[61]	train-logloss:0.341878	valid-logloss:0.354686
[62]	train-logloss:0.341019	valid-logloss:0.354079
[63]	train-logloss:0.340447	valid-logloss:0.353711
[64]	train-logloss:0.339636	valid-logloss:0.353051
[65]	train-logloss:0.338978	valid-logloss:0.352662
[66]	train-logloss:0.33856	valid-logloss:0.352402
[67]	train-logloss:0.338009	valid-logloss:0.352068
[68]	train-logloss:0.337563	valid-logloss:0.35178
[69]	train-logloss:0.336852	valid-logloss:0.351327
[70]	train-logloss:0.33636	valid-logloss:0.350915
[71]	train-logloss:0.335901	valid-logloss:0.35064
[72]	train-logloss:0.335208	valid-logloss:0.350124
[73]	train-logloss:0.33467	valid-logloss:0.349748
[74]	train-logloss:0.334105	valid-logloss:0.349423
[75]	train-logloss:0.333485	valid-logloss:0.349063
[76]	train-logloss:0.333125	valid-logloss:0.348858
[77]	train-logloss:0.332614	valid-logl

[217]	train-logloss:0.285829	valid-logloss:0.3285
[218]	train-logloss:0.285513	valid-logloss:0.328431
[219]	train-logloss:0.285175	valid-logloss:0.328335
[220]	train-logloss:0.284835	valid-logloss:0.328255
[221]	train-logloss:0.28454	valid-logloss:0.32819
[222]	train-logloss:0.28435	valid-logloss:0.328165
[223]	train-logloss:0.284016	valid-logloss:0.328092
[224]	train-logloss:0.283844	valid-logloss:0.328034
[225]	train-logloss:0.283514	valid-logloss:0.32793
[226]	train-logloss:0.283372	valid-logloss:0.327902
[227]	train-logloss:0.283147	valid-logloss:0.327837
[228]	train-logloss:0.282799	valid-logloss:0.327717
[229]	train-logloss:0.282673	valid-logloss:0.327674
[230]	train-logloss:0.28235	valid-logloss:0.327581
[231]	train-logloss:0.282189	valid-logloss:0.327553
[232]	train-logloss:0.281946	valid-logloss:0.327494
[233]	train-logloss:0.281701	valid-logloss:0.327436
[234]	train-logloss:0.281385	valid-logloss:0.327378
[235]	train-logloss:0.281197	valid-logloss:0.327311
[236]	train-logloss

[376]	train-logloss:0.250948	valid-logloss:0.320302
[377]	train-logloss:0.250728	valid-logloss:0.320256
[378]	train-logloss:0.250531	valid-logloss:0.320197
[379]	train-logloss:0.250319	valid-logloss:0.320153
[380]	train-logloss:0.250214	valid-logloss:0.320156
[381]	train-logloss:0.250076	valid-logloss:0.320127
[382]	train-logloss:0.249806	valid-logloss:0.320088
[383]	train-logloss:0.249636	valid-logloss:0.320046
[384]	train-logloss:0.249483	valid-logloss:0.320048
[385]	train-logloss:0.249331	valid-logloss:0.320009
[386]	train-logloss:0.249227	valid-logloss:0.319991
[387]	train-logloss:0.248976	valid-logloss:0.319955
[388]	train-logloss:0.248784	valid-logloss:0.319917
[389]	train-logloss:0.248651	valid-logloss:0.319889
[390]	train-logloss:0.248403	valid-logloss:0.319829
[391]	train-logloss:0.248319	valid-logloss:0.319805
[392]	train-logloss:0.24809	valid-logloss:0.319684
[393]	train-logloss:0.247868	valid-logloss:0.319629
[394]	train-logloss:0.247797	valid-logloss:0.319608
[395]	train-l

In [9]:
result = pd.DataFrame(result, columns=['max_depth', 'train_logloss', 'test_logloss'])

In [11]:
result['diff'] = abs(result.train_logloss - result.test_logloss)

In [14]:
result.sort_values(by=['test_logloss', 'diff'], ascending=(True, True))

,max_depth,train_logloss,test_logloss,diff
5,6,0.246823,0.319379,0.072556
4,5,0.283204,0.326123,0.042920
3,4,0.311500,0.334930,0.023430
2,3,0.332661,0.344782,0.012121
1,2,0.352787,0.359211,0.006424
0,1,0.390950,0.394802,0.003852


Here I am choosing `max_depth` of both 5 and 6. I believe a good param tuning on regularisation would do solve the little overfitting. Now let's fix the number of estimatiors

In [18]:
clf.best_ntree_limit

400

In [6]:
params = {
    'subsample': [0.5, 0.75, 1.0], # Row sampling like Random Forest 
    'colsample_bytree': [0.3, 0.5, 0.7, 1.0],  # Column sampling like Random Forest
}

### For `max_depth = 5`

In [7]:
result = []
for ss in [0.5, 0.75, 1.0]:
    for cs in [0.3, 0.5, 0.7, 1.0]:
        params = {}
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'logloss'
        params['eta'] = 0.1
        params['max_depth'] = 5
        params['silent'] = 1
        params['subsample'] = ss
        params['colsample_bytree'] = cs

        d_train = xgb.DMatrix(X_train, label=ytrain)
        d_test = xgb.DMatrix(X_test, label=ytest)

        watchlist = [(d_train, 'train'), (d_test, 'valid')]

        clf = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=1)
        predict_y = clf.predict(d_test)
        predict_y_train = clf.predict(d_train)
        res = (ss, cs, log_loss(ytrain, predict_y_train, eps=1e-15), log_loss(ytest, predict_y, eps=1e-15))
        result.append(res)
        print(f"Done for subsample = {ss} and colsample_bytree = {cs}")

[0]	train-logloss:0.659275	valid-logloss:0.659542
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.622131	valid-logloss:0.622718
[2]	train-logloss:0.593687	valid-logloss:0.594594
[3]	train-logloss:0.569986	valid-logloss:0.571142
[4]	train-logloss:0.553824	valid-logloss:0.555131
[5]	train-logloss:0.538421	valid-logloss:0.539851
[6]	train-logloss:0.520758	valid-logloss:0.522388
[7]	train-logloss:0.505601	valid-logloss:0.507451
[8]	train-logloss:0.496537	valid-logloss:0.498463
[9]	train-logloss:0.487799	valid-logloss:0.489853
[10]	train-logloss:0.476012	valid-logloss:0.478201
[11]	train-logloss:0.469827	valid-logloss:0.472112
[12]	train-logloss:0.45996	valid-logloss:0.4624
[13]	train-logloss:0.45111	valid-logloss:0.453672
[14]	train-logloss:0.446168	valid-logloss:0.448819
[15]	train-logloss:0.441331	valid-logloss:0.444103
[16]	train-logloss:0.436346	valid-logloss:0.439

[158]	train-logloss:0.326173	valid-logloss:0.342719
[159]	train-logloss:0.326	valid-logloss:0.342608
[160]	train-logloss:0.3258	valid-logloss:0.342518
[161]	train-logloss:0.325543	valid-logloss:0.342349
[162]	train-logloss:0.325278	valid-logloss:0.34217
[163]	train-logloss:0.325054	valid-logloss:0.341996
[164]	train-logloss:0.324793	valid-logloss:0.341841
[165]	train-logloss:0.32457	valid-logloss:0.341724
[166]	train-logloss:0.324439	valid-logloss:0.341648
[167]	train-logloss:0.324251	valid-logloss:0.341589
[168]	train-logloss:0.324053	valid-logloss:0.34144
[169]	train-logloss:0.323837	valid-logloss:0.341332
[170]	train-logloss:0.323595	valid-logloss:0.341185
[171]	train-logloss:0.323424	valid-logloss:0.341117
[172]	train-logloss:0.32323	valid-logloss:0.341052
[173]	train-logloss:0.323031	valid-logloss:0.340996
[174]	train-logloss:0.322785	valid-logloss:0.340839
[175]	train-logloss:0.322626	valid-logloss:0.340736
[176]	train-logloss:0.32242	valid-logloss:0.34066
[177]	train-logloss:0.3

[317]	train-logloss:0.300578	valid-logloss:0.332308
[318]	train-logloss:0.300413	valid-logloss:0.332232
[319]	train-logloss:0.300251	valid-logloss:0.332214
[320]	train-logloss:0.300114	valid-logloss:0.33217
[321]	train-logloss:0.299964	valid-logloss:0.332122
[322]	train-logloss:0.299842	valid-logloss:0.332102
[323]	train-logloss:0.29969	valid-logloss:0.332057
[324]	train-logloss:0.299627	valid-logloss:0.332045
[325]	train-logloss:0.299477	valid-logloss:0.331973
[326]	train-logloss:0.299355	valid-logloss:0.331939
[327]	train-logloss:0.299198	valid-logloss:0.331902
[328]	train-logloss:0.299074	valid-logloss:0.331882
[329]	train-logloss:0.298982	valid-logloss:0.331862
[330]	train-logloss:0.298909	valid-logloss:0.331822
[331]	train-logloss:0.298792	valid-logloss:0.33174
[332]	train-logloss:0.298629	valid-logloss:0.331699
[333]	train-logloss:0.298518	valid-logloss:0.331692
[334]	train-logloss:0.298388	valid-logloss:0.331658
[335]	train-logloss:0.298272	valid-logloss:0.331612
[336]	train-log

[73]	train-logloss:0.348644	valid-logloss:0.357375
[74]	train-logloss:0.348227	valid-logloss:0.357111
[75]	train-logloss:0.347879	valid-logloss:0.356868
[76]	train-logloss:0.347385	valid-logloss:0.35647
[77]	train-logloss:0.346983	valid-logloss:0.356202
[78]	train-logloss:0.346689	valid-logloss:0.356018
[79]	train-logloss:0.346341	valid-logloss:0.355788
[80]	train-logloss:0.345959	valid-logloss:0.355504
[81]	train-logloss:0.345523	valid-logloss:0.355194
[82]	train-logloss:0.344959	valid-logloss:0.354683
[83]	train-logloss:0.344673	valid-logloss:0.3545
[84]	train-logloss:0.344247	valid-logloss:0.354199
[85]	train-logloss:0.343885	valid-logloss:0.353921
[86]	train-logloss:0.34353	valid-logloss:0.353675
[87]	train-logloss:0.343124	valid-logloss:0.353381
[88]	train-logloss:0.34279	valid-logloss:0.353182
[89]	train-logloss:0.342528	valid-logloss:0.353031
[90]	train-logloss:0.342164	valid-logloss:0.35279
[91]	train-logloss:0.341865	valid-logloss:0.352571
[92]	train-logloss:0.341587	valid-log

[232]	train-logloss:0.310065	valid-logloss:0.335677
[233]	train-logloss:0.309888	valid-logloss:0.335643
[234]	train-logloss:0.309705	valid-logloss:0.335581
[235]	train-logloss:0.309539	valid-logloss:0.33553
[236]	train-logloss:0.30937	valid-logloss:0.335491
[237]	train-logloss:0.309205	valid-logloss:0.335463
[238]	train-logloss:0.309071	valid-logloss:0.335403
[239]	train-logloss:0.308906	valid-logloss:0.335358
[240]	train-logloss:0.308806	valid-logloss:0.335314
[241]	train-logloss:0.308693	valid-logloss:0.335277
[242]	train-logloss:0.30857	valid-logloss:0.335229
[243]	train-logloss:0.308381	valid-logloss:0.335143
[244]	train-logloss:0.308235	valid-logloss:0.335124
[245]	train-logloss:0.308107	valid-logloss:0.335088
[246]	train-logloss:0.307974	valid-logloss:0.335022
[247]	train-logloss:0.307845	valid-logloss:0.334977
[248]	train-logloss:0.307735	valid-logloss:0.334982
[249]	train-logloss:0.307623	valid-logloss:0.334963
[250]	train-logloss:0.307416	valid-logloss:0.334843
[251]	train-log

...

[111]	train-logloss:0.335003	valid-logloss:0.348498
[112]	train-logloss:0.334845	valid-logloss:0.348394
[113]	train-logloss:0.334581	valid-logloss:0.348252
[114]	train-logloss:0.334243	valid-logloss:0.348052
[115]	train-logloss:0.333953	valid-logloss:0.34787
[116]	train-logloss:0.333835	valid-logloss:0.347763
[117]	train-logloss:0.333547	valid-logloss:0.347642
[118]	train-logloss:0.333337	valid-logloss:0.34753
[119]	train-logloss:0.333037	valid-logloss:0.347377
[120]	train-logloss:0.332561	valid-logloss:0.347009
[121]	train-logloss:0.332428	valid-logloss:0.346954
[122]	train-logloss:0.332229	valid-logloss:0.346804
[123]	train-logloss:0.331921	valid-logloss:0.346601
[124]	train-logloss:0.331631	valid-logloss:0.346419
[125]	train-logloss:0.331353	valid-logloss:0.346233
[126]	train-logloss:0.331036	valid-logloss:0.346047
[127]	train-logloss:0.33068	valid-logloss:0.345779
[128]	train-logloss:0.330464	valid-logloss:0.345618
[129]	train-logloss:0.330197	valid-logloss:0.345399
[130]	train-log

[270]	train-logloss:0.303103	valid-logloss:0.332694
[271]	train-logloss:0.302928	valid-logloss:0.332621
[272]	train-logloss:0.30275	valid-logloss:0.332555
[273]	train-logloss:0.302554	valid-logloss:0.332483
[274]	train-logloss:0.302403	valid-logloss:0.33246
[275]	train-logloss:0.302185	valid-logloss:0.332335
[276]	train-logloss:0.302019	valid-logloss:0.332274
[277]	train-logloss:0.301821	valid-logloss:0.332176
[278]	train-logloss:0.301744	valid-logloss:0.332165
[279]	train-logloss:0.301605	valid-logloss:0.332089
[280]	train-logloss:0.301532	valid-logloss:0.332064
[281]	train-logloss:0.301411	valid-logloss:0.332056
[282]	train-logloss:0.301263	valid-logloss:0.332001
[283]	train-logloss:0.301148	valid-logloss:0.33196
[284]	train-logloss:0.300957	valid-logloss:0.331884
[285]	train-logloss:0.300871	valid-logloss:0.331857
[286]	train-logloss:0.30067	valid-logloss:0.33178
[287]	train-logloss:0.30048	valid-logloss:0.331732
[288]	train-logloss:0.300421	valid-logloss:0.331723
[289]	train-loglos

[26]	train-logloss:0.384236	valid-logloss:0.389077
[27]	train-logloss:0.382507	valid-logloss:0.387445
[28]	train-logloss:0.38101	valid-logloss:0.386045
[29]	train-logloss:0.379378	valid-logloss:0.38455
[30]	train-logloss:0.377809	valid-logloss:0.383034
[31]	train-logloss:0.3765	valid-logloss:0.381841
[32]	train-logloss:0.375337	valid-logloss:0.380814
[33]	train-logloss:0.374042	valid-logloss:0.379593
[34]	train-logloss:0.372945	valid-logloss:0.378601
[35]	train-logloss:0.371839	valid-logloss:0.377599
[36]	train-logloss:0.371008	valid-logloss:0.376826
[37]	train-logloss:0.370089	valid-logloss:0.376075
[38]	train-logloss:0.369183	valid-logloss:0.375291
[39]	train-logloss:0.36813	valid-logloss:0.374314
[40]	train-logloss:0.367209	valid-logloss:0.373468
[41]	train-logloss:0.36653	valid-logloss:0.372888
[42]	train-logloss:0.365695	valid-logloss:0.372135
[43]	train-logloss:0.365044	valid-logloss:0.371596
[44]	train-logloss:0.36431	valid-logloss:0.370961
[45]	train-logloss:0.363669	valid-logl

[186]	train-logloss:0.316419	valid-logloss:0.337704
[187]	train-logloss:0.316231	valid-logloss:0.337611
[188]	train-logloss:0.316017	valid-logloss:0.337497
[189]	train-logloss:0.315848	valid-logloss:0.337434
[190]	train-logloss:0.3157	valid-logloss:0.337392
[191]	train-logloss:0.315571	valid-logloss:0.337317
[192]	train-logloss:0.315436	valid-logloss:0.337297
[193]	train-logloss:0.31526	valid-logloss:0.337232
[194]	train-logloss:0.31501	valid-logloss:0.337074
[195]	train-logloss:0.314879	valid-logloss:0.337041
[196]	train-logloss:0.314703	valid-logloss:0.336999
[197]	train-logloss:0.314486	valid-logloss:0.336931
[198]	train-logloss:0.314284	valid-logloss:0.336856
[199]	train-logloss:0.31399	valid-logloss:0.336684
[200]	train-logloss:0.313755	valid-logloss:0.336566
[201]	train-logloss:0.313641	valid-logloss:0.336545
[202]	train-logloss:0.31341	valid-logloss:0.336424
[203]	train-logloss:0.313204	valid-logloss:0.336364
[204]	train-logloss:0.312994	valid-logloss:0.336291
[205]	train-loglos

[367]	train-logloss:0.287728	valid-logloss:0.327401
[368]	train-logloss:0.287547	valid-logloss:0.327317
[369]	train-logloss:0.287371	valid-logloss:0.327249
[370]	train-logloss:0.287181	valid-logloss:0.327196
[371]	train-logloss:0.286995	valid-logloss:0.327113
[372]	train-logloss:0.286822	valid-logloss:0.327041
[373]	train-logloss:0.286684	valid-logloss:0.327027
[391]	train-logloss:0.284335	valid-logloss:0.326429
[392]	train-logloss:0.28419	valid-logloss:0.326387
[393]	train-logloss:0.284052	valid-logloss:0.326371
[394]	train-logloss:0.283885	valid-logloss:0.326316
[395]	train-logloss:0.283728	valid-logloss:0.32627
[396]	train-logloss:0.283594	valid-logloss:0.326238
[397]	train-logloss:0.283519	valid-logloss:0.326235
[398]	train-logloss:0.283348	valid-logloss:0.326156
[399]	train-logloss:0.283204	valid-logloss:0.326123
Done for subsample = 1.0 and colsample_bytree = 1.0


In [12]:
result = pd.DataFrame(result, columns=['subsample', 'colsample_bytree', 'train_logloss', 'test_logloss'])

In [14]:
result['diff'] = abs(result.train_logloss - result.test_logloss)

In [16]:
result.sort_values('diff', ascending=True)

,subsample,colsample_bytree,train_logloss,test_logloss,diff
8,1.00,0.3,0.288789,0.327298,0.038509
4,0.75,0.3,0.289042,0.327742,0.038701
0,0.50,0.3,0.290548,0.329282,0.038734
9,1.00,0.5,0.287121,0.326746,0.039625
1,0.50,0.5,0.287174,0.328384,0.041210
5,0.75,0.5,0.285986,0.327354,0.041368
10,1.00,0.7,0.285441,0.327047,0.041606
6,0.75,0.7,0.284881,0.326952,0.042071
2,0.50,0.7,0.285799,0.328626,0.042827
11,1.00,1.0,0.283204,0.326123,0.042920


### Finally trying to remove some probable overfitting

In [18]:
result = []
for ra in [1e-3, 1e-2, 0.1, 1, 100]:
    for rl in [1e-3, 1e-2, 0.1, 1, 100]:
        params = {}
        params['objective'] = 'binary:logistic'
        params['eval_metric'] = 'logloss'
        params['eta'] = 0.1
        params['max_depth'] = 5
        params['silent'] = 1
        params['subsample'] = 1
        params['colsample_bytree'] = 0.3
        params['reg_alpha'] = ra
        params['reg_lambda'] = rl

        d_train = xgb.DMatrix(X_train, label=ytrain)
        d_test = xgb.DMatrix(X_test, label=ytest)

        watchlist = [(d_train, 'train'), (d_test, 'valid')]

        clf = xgb.train(params, d_train, 400, watchlist, early_stopping_rounds=20, verbose_eval=1)
        predict_y = clf.predict(d_test)
        predict_y_train = clf.predict(d_train)
        res = (ra, rl, log_loss(ytrain, predict_y_train, eps=1e-15), log_loss(ytest, predict_y, eps=1e-15))
        result.append(res)
        print(f"Done for reg_alpha = {ra} and reg_lambda = {rl}")

[0]	train-logloss:0.653906	valid-logloss:0.654231
Multiple eval metrics have been passed: 'valid-logloss' will be used for early stopping.

Will train until valid-logloss hasn't improved in 20 rounds.
[1]	train-logloss:0.630493	valid-logloss:0.631067
[2]	train-logloss:0.602246	valid-logloss:0.603074
[3]	train-logloss:0.5751	valid-logloss:0.576239
[4]	train-logloss:0.558353	valid-logloss:0.559672
[5]	train-logloss:0.538197	valid-logloss:0.539773
[6]	train-logloss:0.518833	valid-logloss:0.520578
[7]	train-logloss:0.504413	valid-logloss:0.506235
[8]	train-logloss:0.494162	valid-logloss:0.496086
[9]	train-logloss:0.484976	valid-logloss:0.487061
[10]	train-logloss:0.478716	valid-logloss:0.480922
[11]	train-logloss:0.469202	valid-logloss:0.471488
[12]	train-logloss:0.463367	valid-logloss:0.465755
[13]	train-logloss:0.454382	valid-logloss:0.456938
[14]	train-logloss:0.445592	valid-logloss:0.448315
[15]	train-logloss:0.438518	valid-logloss:0.44141
[16]	train-logloss:0.432795	valid-logloss:0.43

[158]	train-logloss:0.32467	valid-logloss:0.341668
[159]	train-logloss:0.32446	valid-logloss:0.341584
[160]	train-logloss:0.32417	valid-logloss:0.341367
[161]	train-logloss:0.323945	valid-logloss:0.341208
[162]	train-logloss:0.323677	valid-logloss:0.341042
[163]	train-logloss:0.323465	valid-logloss:0.340925
[164]	train-logloss:0.323224	valid-logloss:0.340809
[165]	train-logloss:0.323074	valid-logloss:0.340761
[166]	train-logloss:0.322847	valid-logloss:0.340676
[167]	train-logloss:0.322626	valid-logloss:0.340572
[168]	train-logloss:0.32246	valid-logloss:0.340481
[169]	train-logloss:0.322269	valid-logloss:0.340394
[170]	train-logloss:0.322028	valid-logloss:0.340269
[171]	train-logloss:0.321789	valid-logloss:0.340107
[172]	train-logloss:0.321515	valid-logloss:0.339915
[173]	train-logloss:0.321352	valid-logloss:0.339832
[174]	train-logloss:0.32123	valid-logloss:0.339774
[175]	train-logloss:0.321043	valid-logloss:0.339698
[176]	train-logloss:0.320894	valid-logloss:0.339601
[177]	train-loglo

[317]	train-logloss:0.298714	valid-logloss:0.330515
[318]	train-logloss:0.298537	valid-logloss:0.330462
[319]	train-logloss:0.298445	valid-logloss:0.330421
[320]	train-logloss:0.298329	valid-logloss:0.330379
[321]	train-logloss:0.298168	valid-logloss:0.330322
[322]	train-logloss:0.298094	valid-logloss:0.330291
[323]	train-logloss:0.297965	valid-logloss:0.330257
[324]	train-logloss:0.297901	valid-logloss:0.330245
[325]	train-logloss:0.297801	valid-logloss:0.330232
[326]	train-logloss:0.29765	valid-logloss:0.330206
[327]	train-logloss:0.297499	valid-logloss:0.330143
[328]	train-logloss:0.297336	valid-logloss:0.330054
[329]	train-logloss:0.297194	valid-logloss:0.330027
[330]	train-logloss:0.297049	valid-logloss:0.329964
[331]	train-logloss:0.296937	valid-logloss:0.329927
[332]	train-logloss:0.296839	valid-logloss:0.329887
[333]	train-logloss:0.296689	valid-logloss:0.329842
[334]	train-logloss:0.296555	valid-logloss:0.329816
[335]	train-logloss:0.296402	valid-logloss:0.329782
[336]	train-l

[73]	train-logloss:0.349978	valid-logloss:0.358682
[74]	train-logloss:0.349526	valid-logloss:0.358299
[75]	train-logloss:0.34895	valid-logloss:0.357797
[76]	train-logloss:0.3486	valid-logloss:0.357518
[77]	train-logloss:0.348131	valid-logloss:0.3571
[78]	train-logloss:0.347671	valid-logloss:0.356724
[79]	train-logloss:0.347353	valid-logloss:0.356486
[80]	train-logloss:0.346945	valid-logloss:0.356212
[81]	train-logloss:0.346545	valid-logloss:0.355921
[82]	train-logloss:0.34625	valid-logloss:0.355757
[83]	train-logloss:0.345823	valid-logloss:0.355432
[84]	train-logloss:0.345389	valid-logloss:0.355103
[85]	train-logloss:0.345115	valid-logloss:0.354867
[86]	train-logloss:0.34482	valid-logloss:0.35466
[87]	train-logloss:0.344411	valid-logloss:0.354378
[88]	train-logloss:0.344092	valid-logloss:0.354172
[89]	train-logloss:0.343716	valid-logloss:0.353854
[90]	train-logloss:0.343391	valid-logloss:0.35362
[91]	train-logloss:0.343112	valid-logloss:0.353449
[92]	train-logloss:0.342523	valid-loglos

[232]	train-logloss:0.311097	valid-logloss:0.335084
[233]	train-logloss:0.311009	valid-logloss:0.335037
[234]	train-logloss:0.310877	valid-logloss:0.334989
[235]	train-logloss:0.310678	valid-logloss:0.334897
[236]	train-logloss:0.31058	valid-logloss:0.334885
[237]	train-logloss:0.310423	valid-logloss:0.334829
[238]	train-logloss:0.310191	valid-logloss:0.334694
[239]	train-logloss:0.310067	valid-logloss:0.334599
[240]	train-logloss:0.309954	valid-logloss:0.334568
[241]	train-logloss:0.309837	valid-logloss:0.334554
[242]	train-logloss:0.309658	valid-logloss:0.334476
[243]	train-logloss:0.30951	valid-logloss:0.334375
[244]	train-logloss:0.309284	valid-logloss:0.334272
[245]	train-logloss:0.309143	valid-logloss:0.334197
[246]	train-logloss:0.30897	valid-logloss:0.334122
[247]	train-logloss:0.308801	valid-logloss:0.334078
[248]	train-logloss:0.308622	valid-logloss:0.33401
[249]	train-logloss:0.30846	valid-logloss:0.333959
[250]	train-logloss:0.308232	valid-logloss:0.333816
[251]	train-loglo

...

[32]	train-logloss:0.391062	valid-logloss:0.395105
[33]	train-logloss:0.389715	valid-logloss:0.393854
[34]	train-logloss:0.388527	valid-logloss:0.39274
[35]	train-logloss:0.387437	valid-logloss:0.39167
[36]	train-logloss:0.385611	valid-logloss:0.389941
[37]	train-logloss:0.384376	valid-logloss:0.388729
[38]	train-logloss:0.382976	valid-logloss:0.387372
[39]	train-logloss:0.38201	valid-logloss:0.386463
[40]	train-logloss:0.38025	valid-logloss:0.384791
[41]	train-logloss:0.37931	valid-logloss:0.383859
[42]	train-logloss:0.378445	valid-logloss:0.383028
[43]	train-logloss:0.377626	valid-logloss:0.382217
[44]	train-logloss:0.376662	valid-logloss:0.38129
[45]	train-logloss:0.37609	valid-logloss:0.380773
[46]	train-logloss:0.375578	valid-logloss:0.380306
[47]	train-logloss:0.37472	valid-logloss:0.379461
[48]	train-logloss:0.373868	valid-logloss:0.37866
[49]	train-logloss:0.37306	valid-logloss:0.377896
[50]	train-logloss:0.372553	valid-logloss:0.3774
[51]	train-logloss:0.372078	valid-logloss:0

[192]	train-logloss:0.337049	valid-logloss:0.347509
[193]	train-logloss:0.336914	valid-logloss:0.347415
[194]	train-logloss:0.336811	valid-logloss:0.347353
[195]	train-logloss:0.336706	valid-logloss:0.347277
[196]	train-logloss:0.336581	valid-logloss:0.347204
[197]	train-logloss:0.336464	valid-logloss:0.34714
[198]	train-logloss:0.336358	valid-logloss:0.347086
[199]	train-logloss:0.336231	valid-logloss:0.346961
[200]	train-logloss:0.336133	valid-logloss:0.346897
[201]	train-logloss:0.335943	valid-logloss:0.346751
[202]	train-logloss:0.335769	valid-logloss:0.346611
[203]	train-logloss:0.335666	valid-logloss:0.34654
[204]	train-logloss:0.335566	valid-logloss:0.346481
[205]	train-logloss:0.335474	valid-logloss:0.346419
[206]	train-logloss:0.335354	valid-logloss:0.346325
[207]	train-logloss:0.335267	valid-logloss:0.346258
[208]	train-logloss:0.335113	valid-logloss:0.346118
[209]	train-logloss:0.334983	valid-logloss:0.34603
[210]	train-logloss:0.334913	valid-logloss:0.345994
[211]	train-log

[351]	train-logloss:0.323047	valid-logloss:0.338641
[352]	train-logloss:0.322989	valid-logloss:0.338617
[353]	train-logloss:0.322912	valid-logloss:0.33858
[354]	train-logloss:0.322847	valid-logloss:0.338525
[355]	train-logloss:0.322784	valid-logloss:0.338505
[356]	train-logloss:0.32273	valid-logloss:0.338469
[357]	train-logloss:0.322658	valid-logloss:0.338427
[358]	train-logloss:0.322556	valid-logloss:0.33836
[359]	train-logloss:0.322499	valid-logloss:0.338332
[360]	train-logloss:0.322445	valid-logloss:0.338299
[361]	train-logloss:0.322384	valid-logloss:0.338261
[362]	train-logloss:0.322332	valid-logloss:0.338241
[363]	train-logloss:0.322295	valid-logloss:0.338218
[364]	train-logloss:0.322219	valid-logloss:0.338145
[365]	train-logloss:0.322156	valid-logloss:0.338111
[366]	train-logloss:0.322092	valid-logloss:0.338077
[367]	train-logloss:0.322012	valid-logloss:0.33802
[368]	train-logloss:0.321936	valid-logloss:0.337969
[369]	train-logloss:0.32187	valid-logloss:0.337944
[370]	train-loglo

[108]	train-logloss:0.352643	valid-logloss:0.359785
[109]	train-logloss:0.352331	valid-logloss:0.359506
[110]	train-logloss:0.352034	valid-logloss:0.359222
[111]	train-logloss:0.351744	valid-logloss:0.358943
[112]	train-logloss:0.351493	valid-logloss:0.358748
[113]	train-logloss:0.351273	valid-logloss:0.358546
[114]	train-logloss:0.35106	valid-logloss:0.358372
[115]	train-logloss:0.350876	valid-logloss:0.358214
[116]	train-logloss:0.350693	valid-logloss:0.358063
[117]	train-logloss:0.350501	valid-logloss:0.357944
[118]	train-logloss:0.350356	valid-logloss:0.357816
[119]	train-logloss:0.350141	valid-logloss:0.357638
[120]	train-logloss:0.349926	valid-logloss:0.357492
[121]	train-logloss:0.349748	valid-logloss:0.357378
[122]	train-logloss:0.349562	valid-logloss:0.357205
[123]	train-logloss:0.349379	valid-logloss:0.357091
[124]	train-logloss:0.349197	valid-logloss:0.356917
[125]	train-logloss:0.34902	valid-logloss:0.356763
[126]	train-logloss:0.348814	valid-logloss:0.3566
[127]	train-logl

[267]	train-logloss:0.3311	valid-logloss:0.343626
[268]	train-logloss:0.33102	valid-logloss:0.343568
[269]	train-logloss:0.330929	valid-logloss:0.343499
[270]	train-logloss:0.33083	valid-logloss:0.343428
[271]	train-logloss:0.330743	valid-logloss:0.343359
[272]	train-logloss:0.330671	valid-logloss:0.343321
[273]	train-logloss:0.330605	valid-logloss:0.343273
[274]	train-logloss:0.330531	valid-logloss:0.343229
[275]	train-logloss:0.330439	valid-logloss:0.343171
[276]	train-logloss:0.330352	valid-logloss:0.343131
[277]	train-logloss:0.330277	valid-logloss:0.343066
[278]	train-logloss:0.330203	valid-logloss:0.343025
[279]	train-logloss:0.330125	valid-logloss:0.342994
[280]	train-logloss:0.33005	valid-logloss:0.342954
[281]	train-logloss:0.329977	valid-logloss:0.342899
[282]	train-logloss:0.329891	valid-logloss:0.342851
[283]	train-logloss:0.329809	valid-logloss:0.342789
[284]	train-logloss:0.329729	valid-logloss:0.342729
[285]	train-logloss:0.329634	valid-logloss:0.342689
[286]	train-loglo

In [20]:
result = pd.DataFrame(result, columns=['reg_alpha', 'reg_lambda', 'train_logloss', 'test_logloss'])

In [21]:
result['diff'] = abs(result.train_logloss - result.test_logloss)

In [26]:
result.sort_values(['diff', 'test_logloss'], ascending=(True, True))

,reg_alpha,reg_lambda,train_logloss,test_logloss,diff
24,100.000,100.000,0.321659,0.337932,0.016273
22,100.000,0.100,0.320079,0.336990,0.016911
23,100.000,1.000,0.320056,0.336987,0.016931
21,100.000,0.010,0.319833,0.336870,0.017037
20,100.000,0.001,0.319833,0.336870,0.017037
14,0.100,100.000,0.301278,0.329590,0.028312
19,1.000,100.000,0.301651,0.329975,0.028325
9,0.010,100.000,0.301129,0.329509,0.028380
4,0.001,100.000,0.301259,0.329907,0.028647
8,0.010,1.000,0.289509,0.327552,0.038043


In [21]:
q1_train_vec = tf`idf.transform(Xtrain.question1)
q1_test_vec = tfidf.transform(Xtest.question1)
tfidf_feat_q1 = tfidf.get_feature_names()

In [22]:
q1_train_vec.shape

(283002, 48824)

In [181]:
np.median(m.toarray())

0.0

In [25]:
tfidf.fit(Xtrain.question2)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [26]:
q2_train_vec = tfidf.transform(Xtrain.question2)
q2_test_vec = tfidf.transform(Xtest.question2)
tfidf_feat_q2 = tfidf.get_feature_names()

In [27]:
q2_train_vec.shape

(283002, 44801)

In [28]:
len(tfidf_feat_q2)

44801

In [142]:
model.similar_by_word('coffee')

[('coffees', 0.721267819404602),
 ('gourmet_coffee', 0.7057087421417236),
 ('Coffee', 0.6900454759597778),
 ('o_joe', 0.6891065239906311),
 ('Starbucks_coffee', 0.6874972581863403),
 ('coffee_beans', 0.6749703884124756),
 ('latté', 0.664122462272644),
 ('cappuccino', 0.662549614906311),
 ('brewed_coffee', 0.6621608138084412),
 ('espresso', 0.6616827249526978)]

In [143]:
model.similar_by_word('fruit')

[('fruits', 0.7737188935279846),
 ('cherries', 0.6903519630432129),
 ('berries', 0.6854093074798584),
 ('pears', 0.6825329065322876),
 ('citrus_fruit', 0.6694698333740234),
 ('mango', 0.6631807088851929),
 ('grapes', 0.6511256694793701),
 ('berry', 0.6496502757072449),
 ('peaches', 0.642615556716919),
 ('apple', 0.6410146951675415)]